<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/Jaccard_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Test with dblp-xml

In [3]:
df= pd.read_csv('/content/drive/MyDrive/KnowledgeGraph/Integrated/dblp-xml/publish/dblp-xml_publish_all_00.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,16,17,18,19,22,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df

,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang '
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu'
2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck'
3,3,606483,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Bart Wyns'
4,4,606484,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Moritz Grosse-Wentrup'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S'
423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz'
423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques'
423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'P. F. C. Goncalves'


In [5]:
column_check = ['paperTitle','ISSN','paperType','publishedYear','bookName','publisherName']

In [6]:
import time

def convert_row_to_set_new(row):
    start_time =  time.time()
    data = set()
    for data_column in row:
      if(pd.notna(data_column)):
        data.add(data_column)
    return data

def jaccard_sim_new(row1, row2): 
    data1 = convert_row_to_set_new(row1)
    data2 = convert_row_to_set_new(row2)
    start_time = time.time()
    data_intersect = data1.intersection(data2)
    return float(len(data_intersect)) / (len(data1) + len(data2) - len(data_intersect))


In [7]:
row1 = df.loc[0]
row2 = df.loc[1]
set(row1[column_check])

{2017,
 'Comp. Int. and Neurosc.',
 'Shape Completion Using Deep Boltzmann Machine.',
 'article',
 nan,
 nan}

In [26]:
%%timeit
threshole_similarity = 0.5
df_column = df[column_check]
row1 = df_column.loc[0]
column_new = ['paperTitle','ISSN','paperType','publishedYear','bookName','publisherName','similarity']
all_rows = []

for index,row2 in df_column.iterrows():
  similarity_data = jaccard_sim_new(row1,row2)
  if similarity_data > threshole_similarity:
    paperTitle = row2['paperTitle']
    ISSN = row2['ISSN']
    paperType = row2['paperType']
    publishedYear = row2['publishedYear']
    bookName = row2['bookName']
    publisherName = row2['publisherName']
    all_rows.append([paperTitle,ISSN,paperType,publishedYear,bookName,publisherName,similarity_data])
    # print(row2[column_check])
    # print(similarity_data)

df_compare = pd.DataFrame(all_rows, columns=column_new)

1 loop, best of 5: 39.2 s per loop


จะเห็นว่าจากข้อมูล 423,607 หากเราทำการวนลูป 1 ลูปเพื่อเปรียบเทียบจะใช้เวลาประมาณ 39.2 วินาที

แสดงว่าถ้าหากเราต้องการเปรียบเทียบทุกๆค่าจะต้องวนลูป 423,607 ลูป
หรือก็คือจะต้องใช้เวลาเป็น (423,607 * 39.2)/60/60 = 4,612 ชั่วโมง
ซึ่งเป็นเวลาที่ช้าไป

In [23]:
print(f'{row1} is similarity with:')
df_compare

paperTitle       Shape Completion Using Deep Boltzmann Machine.
ISSN                                                        NaN
paperType                                               article
publishedYear                                              2017
bookName                                Comp. Int. and Neurosc.
publisherName                                               NaN
Name: 0, dtype: object is similarity with:


,paperTitle,ISSN,paperType,publishedYear,bookName,publisherName,similarity
0,Shape Completion Using Deep Boltzmann Machine.,NaN,article,2017,Comp. Int. and Neurosc.,NaN,1.0
1,Shape Completion Using Deep Boltzmann Machine.,NaN,article,2017,Comp. Int. and Neurosc.,NaN,1.0
2,A Novel Multilevel-SVD Method to Improve Multi...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
3,A Novel Multilevel-SVD Method to Improve Multi...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
4,A Novel Multilevel-SVD Method to Improve Multi...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
...,...,...,...,...,...,...,...
77,Selection of the Optimal Algorithm for Real-Ti...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
78,Patch-Based Principal Component Analysis for F...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
79,Patch-Based Principal Component Analysis for F...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6
80,Patch-Based Principal Component Analysis for F...,NaN,article,2017,Comp. Int. and Neurosc.,NaN,0.6


ผลที่ได้จะเห็นว่าข้อมูล DBLP-XML จะมีการแสดงข้อมูลที่มีความคล้ายมากกว่า 0.5 ดังตารางข้างบน ซึ่งจากการดูพบว่ามีข้อมูล ISSN, paperType, PuvlishedYear, bookName, publisherName ที่ตรงกัน หมด

แต่ก็ยังมี PaperTitle ที่แตกต่างกันอยู่ จึงอาจจะต้องลองเพิ่ม Column ในการตรวจสอบดู

## เพิ่ม Column ที่ชื่อว่า DOI

In [29]:
df

,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name,DOI_NEW
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ',"'https://doi.org/10.1155/2017/5705693', 'https..."
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu',"'https://doi.org/10.1155/2017/5705693', 'https..."
2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck',"'https://doi.org/10.1155/2011/217987', 'https:..."
3,3,606483,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Bart Wyns',"'https://doi.org/10.1155/2011/217987', 'https:..."
4,4,606484,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Moritz Grosse-Wentrup',"'https://doi.org/10.1155/2011/217987', 'https:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S','https://doi.org/10.1109/TIE.2018.2818667'
423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz','https://doi.org/10.1109/TIE.2018.2818667'
423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques','https://doi.org/10.1109/TIE.2018.2818667'
423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN

จะเห็นว่า คอลัมน์ DOI เก็บเป็น List เราจึงต้องทำการ Expode ก่อน

In [10]:
def removeBracket(string):
  if len(string) > 0 and string[0] == '[':
    string = string[1:]
  if len(string) > 0 and string[-1] == ']':
    string = string[:-1]
  return string

In [36]:
df_new = df

In [37]:
df_new['DOI_NEW'] = df['DOI'].apply(removeBracket)

In [38]:
df_new

,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name,DOI_NEW
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ',"'https://doi.org/10.1155/2017/5705693', 'https..."
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu',"'https://doi.org/10.1155/2017/5705693', 'https..."
2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck',"'https://doi.org/10.1155/2011/217987', 'https:..."
3,3,606483,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Bart Wyns',"'https://doi.org/10.1155/2011/217987', 'https:..."
4,4,606484,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Moritz Grosse-Wentrup',"'https://doi.org/10.1155/2011/217987', 'https:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S','https://doi.org/10.1109/TIE.2018.2818667'
423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz','https://doi.org/10.1109/TIE.2018.2818667'
423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques','https://doi.org/10.1109/TIE.2018.2818667'
423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN

In [39]:
df_new = df.assign(DOI_NEW=df['DOI_NEW'].str.split(","))

In [40]:
df_new

,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name,DOI_NEW
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ',"['https://doi.org/10.1155/2017/5705693', 'htt..."
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu',"['https://doi.org/10.1155/2017/5705693', 'htt..."
2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck',"['https://doi.org/10.1155/2011/217987', 'http..."
3,3,606483,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Bart Wyns',"['https://doi.org/10.1155/2011/217987', 'http..."
4,4,606484,no,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Moritz Grosse-Wentrup',"['https://doi.org/10.1155/2011/217987', 'http..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S',['https://doi.org/10.1109/TIE.2018.2818667']
423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz',['https://doi.org/10.1109/TIE.2018.2818667']
423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques',['https://doi.org/10.1109/TIE.2018.2818667']
423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,Na

In [41]:
df_new = df_new.explode('DOI_NEW')

In [42]:
df_new

,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name,DOI_NEW
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ','https://doi.org/10.1155/2017/5705693'
0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ','https://www.wikidata.org/entity/Q41214384'
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu','https://doi.org/10.1155/2017/5705693'
1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu','https://www.wikidata.org/entity/Q41214384'
2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck','https://doi.org/10.1155/2011/217987'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S','https://doi.org/10.1109/TIE.2018.2818667'
423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz','https://doi.org/10.1109/TIE.2018.2818667'
423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques','https://doi.org/10.1109/TIE.2018.2818667'
423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Use

In [43]:
df_new[column_check]

,paperTitle,ISSN,DOI_NEW,paperType,publishedYear,bookName,publisherName
0,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://doi.org/10.1155/2017/5705693',article,2017,Comp. Int. and Neurosc.,NaN
0,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://www.wikidata.org/entity/Q41214384',article,2017,Comp. Int. and Neurosc.,NaN
1,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://doi.org/10.1155/2017/5705693',article,2017,Comp. Int. and Neurosc.,NaN
1,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://www.wikidata.org/entity/Q41214384',article,2017,Comp. Int. and Neurosc.,NaN
2,Multisubject Learning for Common Spatial Patte...,NaN,'https://doi.org/10.1155/2011/217987',article,2011,Comp. Int. and Neurosc.,NaN
...,...,...,...,...,...,...,...
423602,Predictive Torque and Rotor Flux Control of a ...,NaN,'https://doi.org/10.1109/TIE.2018.2818667',article,2018,IEEE Trans. Industrial Electronics,NaN
423603,Predictive Torque and Rotor Flux Control of a ...,NaN,'https://doi.org/10.1109/TIE.2018.2818667',article,2018,IEEE Trans. Industrial Electronics,NaN
423604,Predictive Torque and Rotor Flux Control of a ...,NaN,'https://doi.org/10.1109/TIE.2018.2818667',article,2018,IEEE Trans. Industrial Electronics,NaN
423605,Predictive Torque and Rotor Flux Control of a ...,NaN,'https://doi.org/10.1109/TIE.2018.2818667',article,2018,IEEE Trans. Industrial Electronics,NaN


In [50]:
df_new = df_new.reset_index()

In [51]:
df_new

,index,id,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef,clean_name,DOI_NEW
0,0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ','https://doi.org/10.1155/2017/5705693'
1,0,0,606480,yes,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Zheng Wang ','https://www.wikidata.org/entity/Q41214384'
2,1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu','https://doi.org/10.1155/2017/5705693'
3,1,1,606481,no,dblp-xml,journals/cin/WangW17,2197004,Shape Completion Using Deep Boltzmann Machine.,article,NaN,NaN,NaN,5705693:1-5705693:10,2018-11-14,2017,NaN,Comp. Int. and Neurosc.,NaN,2017,NaN,NaN,NaN,"['https://doi.org/10.1155/2017/5705693', 'http...",NaN,db/journals/cin/cin2017.html#WangW17,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Qingbiao Wu','https://www.wikidata.org/entity/Q41214384'
4,2,2,606482,yes,dblp-xml,journals/cin/DevlaminckWGOS11,2197006,Multisubject Learning for Common Spatial Patte...,article,NaN,NaN,NaN,217987:1-217987:9,2018-11-14,2011,NaN,Comp. Int. and Neurosc.,NaN,2011,NaN,NaN,NaN,"['https://doi.org/10.1155/2011/217987', 'https...",NaN,db/journals/cin/cin2011.html#DevlaminckWGOS11,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Dieter Devlaminck','https://doi.org/10.1155/2011/217987'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561552,423602,423602,608529,yes,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'S','https://doi.org/10.1109/TIE.2018.2818667'
561553,423603,423603,774161,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'rgio M. A. Cruz','https://doi.org/10.1109/TIE.2018.2818667'
561554,423604,423604,862058,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667'],NaN,db/journals/tie/tie65.html#CruzMGI18,/Users/polnoy/Documents/app/tnrr-puller/assets...,NaN,'Gil D. Marques','https://doi.org/10.1109/TIE.2018.2818667'
561555,423605,423605,862059,no,dblp-xml,journals/tie/CruzMGI18,290451,Predictive Torque and Rotor Flux Control of a ...,article,NaN,NaN,NaN,9301-9310,2019-07-26,2018,NaN,IEEE Trans. Industrial Electronics,12,65,NaN,NaN,NaN,['https://doi.org/10.1109/TIE.2018.2818667

In [52]:
column_check = ['paperTitle','ISSN','DOI_NEW','paperType','publishedYear','bookName','publisherName']

In [53]:

threshole_similarity = 0.5
df_column = df_new[column_check]
row1 = df_column.loc[0]
column_new = ['paperTitle','ISSN','DOI_NEW','paperType','publishedYear','bookName','publisherName','similarity']
all_rows = []

for index,row2 in df_column.iterrows():
  similarity_data = jaccard_sim_new(row1,row2)
  if similarity_data > threshole_similarity:
    paperTitle = row2['paperTitle']
    issn = row2['ISSN']
    doiNew = row2['DOI_NEW']
    paperType = row2['paperType']
    publishedYear = row2['publishedYear']
    bookName = row2['bookName']
    publisherName = row2['publisherName']
    all_rows.append([paperTitle,issn,doiNew,paperType,publishedYear,bookName,publisherName,similarity_data])
    # print(row2[column_check])
    # print(similarity_data)

df_compare = pd.DataFrame(all_rows, columns=column_new)

In [54]:
print(f'{row1} is similarity with:')
df_compare

paperTitle       Shape Completion Using Deep Boltzmann Machine.
ISSN                                                        NaN
DOI_NEW                  'https://doi.org/10.1155/2017/5705693'
paperType                                               article
publishedYear                                              2017
bookName                                Comp. Int. and Neurosc.
publisherName                                               NaN
Name: 0, dtype: object is similarity with:


,paperTitle,ISSN,DOI_NEW,paperType,publishedYear,bookName,publisherName,similarity
0,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://doi.org/10.1155/2017/5705693',article,2017,Comp. Int. and Neurosc.,NaN,1.000000
1,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://www.wikidata.org/entity/Q41214384',article,2017,Comp. Int. and Neurosc.,NaN,0.666667
2,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://doi.org/10.1155/2017/5705693',article,2017,Comp. Int. and Neurosc.,NaN,1.000000
3,Shape Completion Using Deep Boltzmann Machine.,NaN,'https://www.wikidata.org/entity/Q41214384',article,2017,Comp. Int. and Neurosc.,NaN,0.666667


เมื่อมีการเพิ่ม Column ที่ชื่อว่า DOI ในการตรวจสอบจะพบว่าจะสามารถหา ความ PaperTitle ที่มีความใกล้เคียงมากขึ้น